In [2]:
!pip uninstall -y tensorflow tensorflow-hub
!pip cache purge
!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
Found existing installation: tensorflow-hub 0.16.1
Uninstalling tensorflow-hub-0.16.1:
  Successfully uninstalled tensorflow-hub-0.16.1
Files removed: 12 (237 kB)


In [3]:
# Importing the required libraries and packages

import numpy as np
#!pip install --upgrade tensorflow tensorflow-hub
# TENSORFLOW AND IT'S PACKAGES

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
# Loading the dataset and splitting into different sets required
train_data , test_data , validation_data=tfds.load(name="imdb_reviews" , split=("train[:60%]" , "train[:60%]" , "test" ) , as_supervised=True)


In [5]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [6]:
# Data Exploration
train_example_batch , train_labels_batch =next(iter(train_data.batch(10)))
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [8]:
embedding_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

# Create hub layer with proper initialization
hub_layer = hub.KerasLayer(
    handle=embedding_url,
    input_shape=[],
    dtype=tf.string,
    trainable=True
)

# Build model
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [9]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[ 0.04070194,  0.00420416, -0.01570055, ..., -0.02733925,
         0.07720678, -0.0280134 ],
       [ 0.0230833 ,  0.03450447,  0.03191737, ..., -0.00709982,
         0.09138072,  0.00642445],
       [ 0.02236583, -0.00636588,  0.04310487, ..., -0.04315182,
         0.05988492,  0.02873515]], dtype=float32)>

In [10]:
# Adding different layers to the model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16 , activation ="relu" ))
model.add(tf.keras.layers.Dense(1 , activation="sigmoid"))


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 16)                8208      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 256806049 (979.64 MB)
Trainable params: 256806049 (979.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer="adam" , 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])


In [ ]:
history = model.fit(train_data.shuffle(10000).batch(100) , 
                    epochs =25, validation_data = validation_data.batch(100) ,
                    verbose=1)

Epoch 1/25


C:\Users\mdkai\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 35/150 [======>.......................] - ETA: 1:40 - loss: 0.6127 - accuracy: 0.6866 